<a href="https://colab.research.google.com/github/helenabarmer/60DaysOfUdacity/blob/master/Federated_Learning_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Install PySyft in Google Colab

!pip install tf-encrypted

! URL="https://github.com/openmined/PySyft.git" && FOLDER="PySyft" && if [ ! -d $FOLDER ]; then git clone -b dev --single-branch $URL; else (cd $FOLDER && git pull $URL && cd ..); fi;

!cd PySyft; python setup.py install  > /dev/null

import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
!pip install --upgrade --force-reinstall lz4
!pip install --upgrade --force-reinstall websocket
!pip install --upgrade --force-reinstall websockets
!pip install --upgrade --force-reinstall zstd

From https://github.com/openmined/PySyft
 * branch              HEAD       -> FETCH_HEAD
Already up to date.
zip_safe flag not set; analyzing archive contents...
  Using cached https://files.pythonhosted.org/packages/0a/c6/96bbb3525a63ebc53ea700cc7d37ab9045542d33b4d262d0f0408ad9bbf2/lz4-2.1.10-cp36-cp36m-manylinux1_x86_64.whl
ERROR: syft 0.1.22a1 has requirement msgpack>=0.6.1, but you'll have msgpack 0.5.6 which is incompatible.
ERROR: syft 0.1.22a1 has requirement tf_encrypted!=0.5.7,>=0.5.4, but you'll have tf-encrypted 0.5.7 which is incompatible.
  Found existing installation: lz4 2.1.10
    Uninstalling lz4-2.1.10:
      Successfully uninstalled lz4-2.1.10
  Using cached https://files.pythonhosted.org/packages/bf/45/142141aa47e01a5779f0fa5a53b81f8379ce8f2b1cd13df7d2f1d751ae42/greenlet-0.4.15-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f2/ca/5b5962361ed832847b6b2f9a2d0452c8c2f29a93baef850bb8ad067c7bf9/gevent-1.4.0-cp36-cp36m-manylinux1_x

In [0]:
# Thanks to https://colab.research.google.com/github/chokkan/deeplearningclass/blob/master/mnist.ipynb#scrollTo=Pr_UACmc3Qgc
import gzip
import os
import struct
import numpy as np

def read_image(fi):
    magic, n, rows, columns = struct.unpack(">IIII", fi.read(16))
    assert magic == 0x00000803
    assert rows == 28
    assert columns == 28
    rawbuffer = fi.read()
    assert len(rawbuffer) == n * rows * columns
    rawdata = np.frombuffer(rawbuffer, dtype='>u1', count=n*rows*columns)
    return rawdata.reshape(n, rows, columns).astype(np.float32) / 255.0

def read_label(fi):
    magic, n = struct.unpack(">II", fi.read(8))
    assert magic == 0x00000801
    rawbuffer = fi.read()
    assert len(rawbuffer) == n
    return np.frombuffer(rawbuffer, dtype='>u1', count=n)

if __name__ == '__main__':
    os.system('wget -N http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz')
    os.system('wget -N http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz')
    os.system('wget -N http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz')
    os.system('wget -N http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz')
    
    np.savez_compressed(
        'mnist',
        train_images=read_image(gzip.open('train-images-idx3-ubyte.gz', 'rb')),
        train_labels=read_label(gzip.open('train-labels-idx1-ubyte.gz', 'rb')),
        test_images=read_image(gzip.open('t10k-images-idx3-ubyte.gz', 'rb')),
        test_labels=read_label(gzip.open('t10k-labels-idx1-ubyte.gz', 'rb'))
    )

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import syft as sy

# Hook PyTorch
hook = sy.TorchHook(torch)

# Remote workers
ada = sy.VirtualWorker(hook, id="ada")
helena = sy.VirtualWorker(hook, id="helena")

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

W0804 11:35:14.979645 140337805215616 hook.py:98] Torch was already hooked... skipping hooking process


cpu


In [0]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.lr = 0.01
        self.epochs = 10
        self.seed = 1
        self.no_cuda = False
        self.print_every = 200
        self.log_interval = 30
        self.plot_every = 100
        self.use_cuda = False
        self.save_model = False
        
args = Arguments()
torch.manual_seed(args.seed)
use_cuda = not args.no_cuda and torch.cuda.is_available()

In [18]:
data = np.load('mnist.npz')
print(data['train_images'].shape, data['train_images'].dtype)
print(data['train_labels'].shape, data['train_labels'].dtype)
print(data['test_images'].shape, data['test_images'].dtype)
print(data['test_labels'].shape, data['test_labels'].dtype)

(60000, 28, 28) float32
(60000,) uint8
(10000, 28, 28) float32
(10000,) uint8


In [0]:
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))])

trainset = datasets.MNIST('../data', train=True, download=True, transform=transform)
testset = datasets.MNIST('../data', train=False, download=True, transform=transform)


In [0]:
federated_trainloader = sy.FederatedDataLoader(
    trainset.federate((ada, helena)),
    batch_size=64, shuffle=True)

In [0]:
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [22]:
class Classifier(nn.Module):
  def __init__(self):
    super(Classifier, self).__init__()
    self.fc1 = nn.Linear(784, 250)
    self.fc2 = nn.Linear(250, 100)
    self.fc3 = nn.Linear(100, 10)
    
  def forward (self, x):
    x = x.view(x.shape[0], -1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return F.log_softmax(x, dim =1)
  
model = Classifier()
print(model)

Classifier(
  (fc1): Linear(in_features=784, out_features=250, bias=True)
  (fc2): Linear(in_features=250, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=10, bias=True)
)


In [0]:
def train (args, model, device, federated_trainloader, optimizer, epoch):
  model.train()
  for param in model.parameters():
    param.requires_grad = True
  for batch_idx, (data, target) in enumerate(federated_trainloader):
    model.send(data.location)
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    model.get()
    if batch_idx % args.log_interval == 0:
        loss = loss.get()
        
        print('Train epoch: {} [{} / {} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * args.batch_size, len(federated_trainloader) * args.batch_size,
        100. * batch_idx / len(federated_trainloader), loss.item()))

In [0]:
def test(args, model, device, testloader):
  model.eval()
  test_loss = 0
  correct = 0
  
  with torch.no_grad():
    for data, target in testloader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      test_loss += F.nll_loss(output, target, reduction='sum').item()
      pred = output.argmax(1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()
      
  test_loss /= len(testloader.dataset)
  
  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
  test_loss, correct, len(testloader.dataset),
  100. * correct /len(testloader.dataset)))

In [25]:
%%time
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr)

for epoch in range(1, args.epochs + 1):
  train(args, model, device, federated_trainloader, optimizer, epoch)
  test(args, model, device, testloader)
  
if (args.save_model):
  torch.save(model.state_dict(), "mnist_cn.pt")

Train epoch: 1 [0 / 60032 (0%)]	Loss: 2.283067
Train epoch: 1 [1920 / 60032 (3%)]	Loss: 2.219810
Train epoch: 1 [3840 / 60032 (6%)]	Loss: 2.037053
Train epoch: 1 [5760 / 60032 (10%)]	Loss: 1.890976
Train epoch: 1 [7680 / 60032 (13%)]	Loss: 1.698693
Train epoch: 1 [9600 / 60032 (16%)]	Loss: 1.358035
Train epoch: 1 [11520 / 60032 (19%)]	Loss: 1.161821
Train epoch: 1 [13440 / 60032 (22%)]	Loss: 0.929798
Train epoch: 1 [15360 / 60032 (26%)]	Loss: 0.780834
Train epoch: 1 [17280 / 60032 (29%)]	Loss: 0.743581
Train epoch: 1 [19200 / 60032 (32%)]	Loss: 0.609117
Train epoch: 1 [21120 / 60032 (35%)]	Loss: 0.553566
Train epoch: 1 [23040 / 60032 (38%)]	Loss: 0.783444
Train epoch: 1 [24960 / 60032 (42%)]	Loss: 0.620301
Train epoch: 1 [26880 / 60032 (45%)]	Loss: 0.405261
Train epoch: 1 [28800 / 60032 (48%)]	Loss: 0.552524
Train epoch: 1 [30720 / 60032 (51%)]	Loss: 0.391389
Train epoch: 1 [32640 / 60032 (54%)]	Loss: 0.316816
Train epoch: 1 [34560 / 60032 (58%)]	Loss: 0.281896
Train epoch: 1 [36480 / 